In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd


data_23h = pd.read_csv("/Users/adwge99/Desktop/Cours/Innovation e recherche/Mudlogging/POC-Forage/DATA/Data Export_CORIANCE_GAL3_13022025-23h-23h59.csv")
data_23h

,TIME,DBTM,DMEA,SPPA,BPOS,SPM1,SPM2,MFIA,WOBA,HKLA,RPMA,TQA,TV01,TV02,TV03,TV04,TV05,TV06,TTV1,ROPA
0,2025-02-13 23:00:01,18.70372,222.2,3.0,22.5759,0.0,0.0,0.0,0.0,12.9510,0.0,0.0,8.24450,40.3728,27.4979,15.0503,15.3227,-0.450772,0.000521,0.0
1,2025-02-13 23:00:04,18.70372,222.2,0.0,21.3725,0.0,0.0,0.0,0.0,13.2303,0.0,0.0,8.24450,40.3446,27.4999,15.0493,15.3237,-0.455047,0.000521,0.0
2,2025-02-13 23:00:08,18.70372,222.2,1.0,18.9832,0.0,0.0,0.0,0.0,11.1075,0.0,0.0,8.24103,40.3414,27.5160,15.0493,15.3247,-0.450772,0.000521,0.0
3,2025-02-13 23:00:11,18.70372,222.2,2.0,16.5004,0.0,0.0,0.0,0.0,11.6661,0.0,0.0,8.23325,40.3477,27.5140,15.0493,15.3237,-0.459322,0.000000,0.0
4,2025-02-13 23:00:15,18.70372,222.2,3.0,15.2561,0.0,0.0,0.0,0.0,13.7889,0.0,0.0,8.23238,40.3446,27.4999,15.0503,15.3237,-0.459322,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,2025-02-13 23:59:42,18.70372,222.2,3.0,24.6264,0.0,0.0,0.0,0.0,13.1186,0.0,0.0,8.22546,40.3854,27.3185,15.0615,15.3217,21.270300,0.000521,0.0
997,2025-02-13 23:59:45,18.70372,222.2,5.0,24.6264,0.0,0.0,0.0,0.0,12.7275,0.0,0.0,8.21853,40.3854,27.3225,15.0635,15.3217,21.270300,0.000521,0.0
998,2025-02-13 23:59:49,18.70372,222.2,4.0,24.6264,0.0,0.0,0.0,0.0,13.5096,0.0,0.0,8.21853,40.3885,27.3185,15.0615,15.3217,21.274500,0.000000,0.0
999,2025-02-13 23:59:52,18.70372,222.2,4.0,24.6264,0.0,0.0,0.0,0.0,14.7945,0.0,0.0,8.21853,40.3854,27.3205,15.0544,15.3227,21.270300,0.000000,0.0


In [4]:
data_23h.columns

Index(['TIME', 'DBTM', 'DMEA', 'SPPA', 'BPOS', 'SPM1', 'SPM2', 'MFIA', 'WOBA',
       'HKLA', 'RPMA', 'TQA', 'TV01', 'TV02', 'TV03', 'TV04', 'TV05', 'TV06',
       'TTV1', 'ROPA'],
      dtype='object')

In [5]:
df = data_23h

In [8]:
import pandas as pd

# Fonction pour déterminer les états
def determiner_etat(row, previous_bpos):
    etats = []

    # Vérifier le mouvement vertical du trépan
    if row["BPOS"] > previous_bpos:
        if row["HKLA"] < 20:
            etats.append("Montée à vide")
        else:
            etats.append("Montée en charge")
    elif row["BPOS"] < previous_bpos:
        if row["HKLA"] < 20:
            etats.append("Descente à vide")
        else:
            etats.append("Descente en charge")

    # Vérifier le forage
    if row["WOBA"] > 0:
        etats.append("Forage")

    # Vérifier la circulation des pompes
    if row["SPM1"] > 0 and row["SPM2"] > 0 and row["SPPA"] > 6 and row["MFIA"] > 0:
        etats.append("Circulation active")
    
    # Si aucun état détecté, considérer "Standby"
    if not etats:
        etats.append("Standby")

    return " + ".join(etats)  # Retourner les états combinés sous forme de chaîne

# Initialiser l'état du trépan
previous_bpos = df["BPOS"].iloc[0]
df["État"] = [determiner_etat(row, previous_bpos) for index, row in df.iterrows()]
previous_bpos = df["BPOS"].iloc[-1]

# Sauvegarde du fichier modifié
output_path = "Data_avec_Etats.csv"
df.to_csv(output_path, index=False)

print(f"Fichier sauvegardé avec les états : {output_path}")

Fichier sauvegardé avec les états : Data_avec_Etats.csv


In [10]:
# Initialiser la valeur précédente de BPOS
previous_bpos = df["BPOS"].iloc[0]

# Définition des événements
def get_mouvement(row, previous_bpos):
    """Détermine le mouvement du trépan en fonction de BPOS et HKLA."""
    if row["BPOS"] > previous_bpos:
        return "Montée à vide" if row["HKLA"] < 20 else "Montée en charge"
    elif row["BPOS"] < previous_bpos:
        return "Descente à vide" if row["HKLA"] < 20 else "Descente en charge"
    return "Stable"

def get_forage(row):
    """Détermine si le trépan fore ou non en fonction de WOBA."""
    return "Forage" if row["WOBA"] > 0 else "Pas de forage"

def get_circulation(row):
    """Détermine si la circulation des pompes est active."""
    if row["SPM1"] > 0 and row["SPM2"] > 0 and row["SPPA"] > 6 and row["MFIA"] > 0:
        return "Circulation active"
    return "Pas de circulation"

# Création des nouvelles colonnes
mouvement_list = []
forage_list = []
circulation_list = []

for index, row in df.iterrows():
    mouvement_list.append(get_mouvement(row, previous_bpos))
    forage_list.append(get_forage(row))
    circulation_list.append(get_circulation(row))
    previous_bpos = row["BPOS"]  # Mise à jour du BPOS précédent

df["Mouvement"] = mouvement_list
df["Forage"] = forage_list
df["Circulation"] = circulation_list

# Sauvegarde du fichier modifié
output_path = "Data_avec_Evenements.csv"
df.to_csv(output_path, index=False)

print(f"Fichier sauvegardé avec les événements : {output_path}")

Fichier sauvegardé avec les événements : Data_avec_Evenements.csv


In [ ]:
df

In [4]:
%pip install pyarrow fastparquet

  Using cached pyarrow-19.0.1-cp313-cp313-macosx_12_0_arm64.whl.metadata (3.3 kB)
Using cached pyarrow-19.0.1-cp313-cp313-macosx_12_0_arm64.whl (30.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.8/683.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
file_path = "DATA/Anatole_processed_concatenated_plusEtat.parquet"
df_Anatole = pd.read_parquet(file_path)
dfexp = df_Anatole.head()

In [11]:
dfexp.columns

Index(['TIME', 'DBTM', 'DMEA', 'SPPA', 'BPOS', 'SPM1', 'SPM2', 'MFIA', 'WOBA',
       'HKLA', 'RPMA', 'TQA', 'TV01', 'TV02', 'TV03', 'TV04', 'TV05', 'TV06',
       'TTV1', 'ROPA', 'DERIVBPOS', 'ASCBPOS', 'DESCBPOS', 'CONSTBPOS',
       'CHARGE', 'FORAGE', 'CIRCULATION', 'str_etat'],
      dtype='object')